In [1]:
import plotly.offline as pyo
import plotly.graph_objs as go
from sklearn.model_selection import KFold
# Importando modulos necessarios para o trabalho
import matplotlib as mpl
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from IPython.display import display
from IPython.display import Markdown

from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from keras import datasets
from keras import layers
from keras import optimizers
from keras import models

# Tira os warning do TF
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

def printmd(s):
    display(Markdown(s))

/home/pedro/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [2]:
data = pd.read_csv('./final_data.csv', index_col=0)

df = data.drop(['sec_app_earliest_cr_line', 'sub_grade', 'addr_state',
                'payment_plan_start_date', 'debt_settlement_flag_date',
                'settlement_date', 'hardship_end_date', 'hardship_start_date'], axis=1)

numcols = df.dtypes[(df.dtypes == 'int64') | (df.dtypes == 'float64')].index
catcols = df.dtypes[(df.dtypes != 'int64') & (df.dtypes != 'float64')].index

/home/pedro/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning:

Columns (13,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
# Definindo a função de normalização
def normalization(x):
    xn = (x-x.mean())/x.std()
    return xn

# Aplicando a normalização aos dados
ndf = normalization(df[numcols])

In [ ]:
# One-hot encoding das variaveis categoricas
cat_vars = pd.concat([pd.get_dummies(df[cat]) for cat in df[catcols].columns.tolist()])
    
    

In [ ]:
df_n = pd.concat([df[catcols], ndf], axis=1)

In [ ]:
X = df_n.drop('loan_status', axis=1).reset_index(drop=True)
y = df_n['loan_status'].reset_index(drop=True)
result = []
for i in y.index:
    if y[i] == 'FullyPaid':
        result.append(0)
    else:
        result.append(1)

y = np.array(result)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

def generate_model(classifier, X, y, params, n_splits):
    cv = KFold(n_splits=n_splits, random_state=1, shuffle=False)
    acc_test_mean = []
    acc_train_mean = []
    acc_test_sd = []
    acc_train_sd = []
    for par in params:
        acc_test = []
        acc_train = []
        for train_index, test_index in cv.split(X):
            # Separando em treino e teste
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            if classifier == 'RF':
                model = RandomForestClassifier(n_estimators=par, max_depth=6)
            elif classifier == 'SVM':
                model = SVC(kernel=par)
            elif classifier == 'DT':
                model = DecisionTreeClassifier(max_depth=par)
            elif classifier == 'KNN':
                model = KNeighborsClassifier(n_neighbors=par)
            
            # Treinando o modelo
            model.fit(X_train, y_train)

            # Fazendo previsões
            y_pred_test = model_rf.predict(X_test)
            y_pred_train = model_rf.predict(X_train)

            # Calculando a acurácia, precisão e recall
            acc_test.append(metrics.accuracy_score(y_test, y_pred_test))
            acc_train.append(metrics.accuracy_score(y_train, y_pred_train))

            # Calculando a média da acurácia
            acc_test_mean.append(np.array(acc_test).mean())
            acc_train_mean.append(np.array(acc_train).mean())
            acc_test_sd.append(np.array(acc_test).std())
            acc_train_sd.append(np.array(acc_test).std())

    return acc_train_mean, acc_train_sd, acc_test_mean, acc_test_sd

In [ ]:
train_mean, train_sd, test_mean, test_sd = generate_model(classifier='RF', X=X, y=y, params=[3,4,5], n_splits=5)

In [ ]:
# Calculando a acurácia, precisão e recall
print('Acurácia Train:', metrics.accuracy_score(train_y, y_pred_train))
print('Acurácia Test:', metrics.accuracy_score(test_y, y_pred_test))
print('Precision:', metrics.precision_score(test_y, y_pred_test))
print('Recall:', metrics.recall_score(test_y, y_pred_test))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
model = Sequential()
model.add(Dense())
model.add(...)
model.add(...)